In [3]:
import tensorflow as tf
import os
import random
import math
import sys
import numpy as np
from PIL import  Image #PIL是python中专门用来处理图片的一个包

In [4]:
#验证集数量
_NUM_TEST = 500

#随机种子
_RANDOM_SEED = 0

#数据集路径
DATASET_DIR = '/Users/yangming/Downloads/jupyter_notebook/captcha_code/captcha/images'
#tfrecord文件存放路径
TFRECORD_DIR = '/Users/yangming/Downloads/jupyter_notebook/captcha_code/captcha/'

#判断tfrecord文件是否存在，存在返回True，否则返回False
def _dataset_exists(dataset_dir):
    for split_name in ['train', 'test']:
        output_filename = os.path.join(dataset_dir, split_name + '.tfrecords')
        if not tf.gfile.Exists(output_filename):
            return False
    return True

#获取所有验证码图片
def _get_filenames_and_classes(dataset_dir):
    photo_filenames = []
    for filename in os.listdir(dataset_dir):
        #获取文件路径
        if filename != '.DS_Store' and filename != '.ipynb_checkpoints': #这一句是自己加的，免得受到macOS系统和jupyter的影响
            path = os.path.join(dataset_dir, filename)
            photo_filenames.append(path)
    return photo_filenames

def int64_feature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value = [values]))

def image_to_tfexample(image_data,label0,label1,label2,label3):
    return tf.train.Example(features=tf.train.Features(feature={
     'image': bytes_feature(image_data),
        'label0':int64_feature(label0),
        'label1':int64_feature(label1),
        'label2':int64_feature(label2),
        'label3':int64_feature(label3)
    }))


#把数据转换为TFRecord格式
def _convert_dataset(split_name, filenames, dataset_dir):
    #如果输入的split_name 不为‘train’或者‘test’，就会报错
    assert split_name in ['train', 'test']
    
    with tf.Session() as sess:
        #定义tfrecord文件的路径和名字
        output_filename = os.path.join(dataset_dir, split_name + '.tfrecords')
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            #使用enumerate遍历，返回下标和元素值
            for i,filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image %d/%d' % (i+1, len(filenames)))
                    sys.stdout.flush()
                    
                    #读取图片
                    image_data = Image.open(filename)
                    #根据模型的结构resize图片
                    image_data = image_data.resize((224, 224))
                    #转化为灰度图
                    image_data = np.array(image_data.convert('L'))
                    #将图片转化为bytes
                    image_data = image_data.tobytes()
                    
                    #获取lable
                    labels = filename.split('/')[-1][0:4]
                    num_labels = []
                    for j in range(4):
                        num_labels.append(int(labels[j]))
                    
                    #生成protocol数据类型（此处并没有看懂）
                    example = image_to_tfexample(image_data, int(labels[0]), int(labels[1]), int(labels[2]), int(labels[3]))
                    tfrecord_writer.write(example.SerializeToString())
                except IOError as e:
                    print("Could not read: ", filename)
                    print('Error: ', e)
                    print('Skip it\n')
    sys.stdout.write('\n')
    sys.stdout.flush()
                    
if _dataset_exists(TFRECORD_DIR):
    print('tfrecord文件已存在')
else:
    #获得所有图片
    photo_filenames = _get_filenames_and_classes(DATASET_DIR)
    
    #把数据集切分成训练集和测试集并打乱
    random.seed(_RANDOM_SEED)
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[_NUM_TEST:]
    testing_filenames = photo_filenames[:_NUM_TEST]

    #数据转换
    _convert_dataset('train', training_filenames, TFRECORD_DIR)
    _convert_dataset('test', testing_filenames, TFRECORD_DIR)
    print("生成tfrecord文件")
    

>> Converting image 4618/4618
>> Converting image 500/500
生成tfrecord文件
